In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.python.keras.layers import Dense
from numpy.random import default_rng
from sklearn.model_selection import train_test_split
import math

In [ ]:
old_full_data = pd.read_csv('data/full_data_10HZ.csv', sep=",", header=None, names=["e_t_x","e_t_y","e_t_z","e_r_x","e_r_y","e_r_z","e_r_w","d_t_x","d_t_y","d_t_z","d_r_x","d_r_y","d_r_z","d_r_w", "c_t_x","c_t_y","c_t_z","c_r_x","c_r_y","c_r_z", "c_r_w",  "speed"])
old_full_data
old_full_data.tail()


In [ ]:
#define function to swap columns
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df


In [ ]:

 
def euler_from_quaternion(x, y, z, w):
        """
        Convert a quaternion into euler angles (roll, pitch, yaw)
        roll is rotation around x in radians (counterclockwise)
        pitch is rotation around y in radians (counterclockwise)
        yaw is rotation around z in radians (counterclockwise)
        """
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll_x = math.atan2(t0, t1)
     
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch_y = math.asin(t2)
     
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw_z = math.atan2(t3, t4)
     
        return roll_x, pitch_y, yaw_z # in radians

In [ ]:

def get_quaternion_from_euler(roll, pitch, yaw):
    qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
    qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
    qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
 
    return qx, qy, qz, qw

In [ ]:
data_stop_0 = old_full_data.loc[old_full_data['speed']=="stopped"][1:].reset_index(drop=True)
data_stop_0

In [ ]:
data_stop_90_left_foot_without_negativation = swap_columns(data_stop_0, 'e_t_x', 'e_t_z')
data_stop_90_left_foot_without_negativation["e_t_z"] = data_stop_90_left_foot_without_negativation["e_t_z"]*-1
data_stop_90_left_foot=data_stop_90_left_foot_without_negativation
data_stop_90_left_right_foot_without_negativation = swap_columns(data_stop_90_left_foot, 'd_t_x', 'd_t_z')
data_stop_90_left_right_foot_without_negativation["d_t_z"] = data_stop_90_left_right_foot_without_negativation["d_t_z"]*-1
data_stop_90=data_stop_90_left_right_foot_without_negativation

data_stop_90.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_stop_90.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_stop_90.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_stop_90.rename(columns={'d_t_z':'d_t_x2'}, inplace=True)
data_stop_90.rename(columns={'d_t_x':'d_t_z'}, inplace=True)
data_stop_90.rename(columns={'d_t_x2':'d_t_x'}, inplace=True)
data_stop_90

In [ ]:
data_stop_180_left_foot_without_negativation = swap_columns(data_stop_90, 'e_t_x', 'e_t_z')
data_stop_180_left_foot_without_negativation["e_t_z"] = data_stop_180_left_foot_without_negativation["e_t_z"]*-1
data_stop_180_left_foot=data_stop_180_left_foot_without_negativation
data_stop_180_left_right_foot_without_negativation = swap_columns(data_stop_180_left_foot, 'd_t_x', 'd_t_z')
data_stop_180_left_right_foot_without_negativation["d_t_z"] = data_stop_180_left_right_foot_without_negativation["d_t_z"]*-1
data_stop_180=data_stop_180_left_right_foot_without_negativation

data_stop_180.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_stop_180.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_stop_180.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_stop_180.rename(columns={'d_t_z':'d_t_x2'}, inplace=True)
data_stop_180.rename(columns={'d_t_x':'d_t_z'}, inplace=True)
data_stop_180.rename(columns={'d_t_x2':'d_t_x'}, inplace=True)
data_stop_180

In [ ]:
data_stop_270_left_foot_without_negativation = swap_columns(data_stop_180, 'e_t_x', 'e_t_z')
data_stop_270_left_foot_without_negativation["e_t_z"] = data_stop_270_left_foot_without_negativation["e_t_z"]*-1
data_stop_270_left_foot=data_stop_270_left_foot_without_negativation
data_stop_270_left_right_foot_without_negativation = swap_columns(data_stop_270_left_foot, 'd_t_x', 'd_t_z')
data_stop_270_left_right_foot_without_negativation["d_t_z"] = data_stop_270_left_right_foot_without_negativation["d_t_z"]*-1
data_stop_270=data_stop_270_left_right_foot_without_negativation

data_stop_270.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_stop_270.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_stop_270.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_stop_270.rename(columns={'d_t_z':'d_t_x2'}, inplace=True)
data_stop_270.rename(columns={'d_t_x':'d_t_z'}, inplace=True)
data_stop_270.rename(columns={'d_t_x2':'d_t_x'}, inplace=True)
data_stop_270

In [ ]:
data_stop_90.loc[:,["d_r_x","d_r_y","d_r_z","d_r_w" ]]

In [ ]:
for i in data_stop_90:
    print(i)

In [ ]:
l_e_r_x_quat_data_stop_90=[]
l_e_r_y_quat_data_stop_90=[]
l_e_r_z_quat_data_stop_90=[]
l_e_r_w_quat_data_stop_90=[]
l_d_r_x_quat_data_stop_90=[]
l_d_r_y_quat_data_stop_90=[]
l_d_r_z_quat_data_stop_90=[]
l_d_r_w_quat_data_stop_90=[]
for index_90, row_90 in data_stop_90.iterrows():
    e_r_x_euler_data_stop_90,e_r_y_euler_data_stop_90,e_r_z_euler_data_stop_90=euler_from_quaternion(row_90[3],row_90[4],row_90[5],row_90[6])
    d_r_x_euler_data_stop_90,d_r_y_euler_data_stop_90,d_r_z_euler_data_stop_90=euler_from_quaternion(row_90[10],row_90[11],row_90[12],row_90[13])
    
    e_r_z_euler_data_stop_90=e_r_z_euler_data_stop_90+np.pi/2
    d_r_z_euler_data_stop_90=d_r_z_euler_data_stop_90+np.pi/2
    
    e_r_x_data_stop_90, e_r_y_data_stop_90, e_r_z_data_stop_90,e_r_w_data_stop_90=get_quaternion_from_euler(e_r_x_euler_data_stop_90, e_r_y_euler_data_stop_90,e_r_z_euler_data_stop_90)
    d_r_x_data_stop_90, d_r_y_data_stop_90, d_r_z_data_stop_90,d_r_w_data_stop_90=get_quaternion_from_euler(d_r_x_euler_data_stop_90, d_r_y_euler_data_stop_90,d_r_z_euler_data_stop_90)
    
    l_e_r_x_quat_data_stop_90.append(e_r_x_data_stop_90)
    l_e_r_y_quat_data_stop_90.append(e_r_y_data_stop_90)
    l_e_r_z_quat_data_stop_90.append(e_r_z_data_stop_90)
    l_e_r_w_quat_data_stop_90.append(e_r_w_data_stop_90)
    l_d_r_x_quat_data_stop_90.append(d_r_x_data_stop_90)
    l_d_r_y_quat_data_stop_90.append(d_r_y_data_stop_90)
    l_d_r_z_quat_data_stop_90.append(d_r_z_data_stop_90)
    l_d_r_w_quat_data_stop_90.append(d_r_w_data_stop_90)
    
data_stop_90['e_r_x']=l_e_r_x_quat_data_stop_90
data_stop_90['e_r_y']=l_e_r_y_quat_data_stop_90
data_stop_90['e_r_z']=l_e_r_z_quat_data_stop_90
data_stop_90['e_r_w']=l_e_r_w_quat_data_stop_90
data_stop_90['d_r_x']=l_d_r_x_quat_data_stop_90
data_stop_90['d_r_y']=l_d_r_y_quat_data_stop_90
data_stop_90['d_r_z']=l_d_r_z_quat_data_stop_90
data_stop_90['d_r_w']=l_d_r_w_quat_data_stop_90


In [ ]:
data_stop_90

In [ ]:
data_stop_90.loc[:,["d_r_x","d_r_y","d_r_z","d_r_w" ]]

In [ ]:
l_e_r_x_quat_data_stop_180=[]
l_e_r_y_quat_data_stop_180=[]
l_e_r_z_quat_data_stop_180=[]
l_e_r_w_quat_data_stop_180=[]
l_d_r_x_quat_data_stop_180=[]
l_d_r_y_quat_data_stop_180=[]
l_d_r_z_quat_data_stop_180=[]
l_d_r_w_quat_data_stop_180=[]
for index_180, row_180 in data_stop_180.iterrows():
    e_r_x_euler_data_stop_180,e_r_y_euler_data_stop_180,e_r_z_euler_data_stop_180=euler_from_quaternion(row_180[3],row_180[4],row_180[5],row_180[6])
    d_r_x_euler_data_stop_180,d_r_y_euler_data_stop_180,d_r_z_euler_data_stop_180=euler_from_quaternion(row_180[10],row_180[11],row_180[12],row_180[13])
    
    e_r_z_euler_data_stop_180=e_r_z_euler_data_stop_180+np.pi
    d_r_z_euler_data_stop_180=d_r_z_euler_data_stop_180+np.pi
    
    e_r_x_data_stop_180, e_r_y_data_stop_180, e_r_z_data_stop_180,e_r_w_data_stop_180=get_quaternion_from_euler(e_r_x_euler_data_stop_180, e_r_y_euler_data_stop_180,e_r_z_euler_data_stop_180)
    d_r_x_data_stop_180, d_r_y_data_stop_180, d_r_z_data_stop_180,d_r_w_data_stop_180=get_quaternion_from_euler(d_r_x_euler_data_stop_180, d_r_y_euler_data_stop_180,d_r_z_euler_data_stop_180)
    
    l_e_r_x_quat_data_stop_180.append(e_r_x_data_stop_180)
    l_e_r_y_quat_data_stop_180.append(e_r_y_data_stop_180)
    l_e_r_z_quat_data_stop_180.append(e_r_z_data_stop_180)
    l_e_r_w_quat_data_stop_180.append(e_r_w_data_stop_180)
    l_d_r_x_quat_data_stop_180.append(d_r_x_data_stop_180)
    l_d_r_y_quat_data_stop_180.append(d_r_y_data_stop_180)
    l_d_r_z_quat_data_stop_180.append(d_r_z_data_stop_180)
    l_d_r_w_quat_data_stop_180.append(d_r_w_data_stop_180)
    
data_stop_180['e_r_x']=l_e_r_x_quat_data_stop_180
data_stop_180['e_r_y']=l_e_r_y_quat_data_stop_180
data_stop_180['e_r_z']=l_e_r_z_quat_data_stop_180
data_stop_180['e_r_w']=l_e_r_w_quat_data_stop_180
data_stop_180['d_r_x']=l_d_r_x_quat_data_stop_180
data_stop_180['d_r_y']=l_d_r_y_quat_data_stop_180
data_stop_180['d_r_z']=l_d_r_z_quat_data_stop_180
data_stop_180['d_r_w']=l_d_r_w_quat_data_stop_180


In [ ]:
data_stop_180

In [ ]:
l_e_r_x_quat_data_stop_270=[]
l_e_r_y_quat_data_stop_270=[]
l_e_r_z_quat_data_stop_270=[]
l_e_r_w_quat_data_stop_270=[]
l_d_r_x_quat_data_stop_270=[]
l_d_r_y_quat_data_stop_270=[]
l_d_r_z_quat_data_stop_270=[]
l_d_r_w_quat_data_stop_270=[]
for index_270, row_270 in data_stop_270.iterrows():
    e_r_x_euler_data_stop_270,e_r_y_euler_data_stop_270,e_r_z_euler_data_stop_270=euler_from_quaternion(row_270[3],row_270[4],row_270[5],row_270[6])
    d_r_x_euler_data_stop_270,d_r_y_euler_data_stop_270,d_r_z_euler_data_stop_270=euler_from_quaternion(row_270[10],row_270[11],row_270[12],row_270[13])
    
    e_r_z_euler_data_stop_270=e_r_z_euler_data_stop_270+np.pi*3/4
    d_r_z_euler_data_stop_270=d_r_z_euler_data_stop_270+np.pi*3/4
    
    e_r_x_data_stop_270, e_r_y_data_stop_270, e_r_z_data_stop_270,e_r_w_data_stop_270=get_quaternion_from_euler(e_r_x_euler_data_stop_270, e_r_y_euler_data_stop_270,e_r_z_euler_data_stop_270)
    d_r_x_data_stop_270, d_r_y_data_stop_270, d_r_z_data_stop_270,d_r_w_data_stop_270=get_quaternion_from_euler(d_r_x_euler_data_stop_270, d_r_y_euler_data_stop_270,d_r_z_euler_data_stop_270)
    
    l_e_r_x_quat_data_stop_270.append(e_r_x_data_stop_270)
    l_e_r_y_quat_data_stop_270.append(e_r_y_data_stop_270)
    l_e_r_z_quat_data_stop_270.append(e_r_z_data_stop_270)
    l_e_r_w_quat_data_stop_270.append(e_r_w_data_stop_270)
    l_d_r_x_quat_data_stop_270.append(d_r_x_data_stop_270)
    l_d_r_y_quat_data_stop_270.append(d_r_y_data_stop_270)
    l_d_r_z_quat_data_stop_270.append(d_r_z_data_stop_270)
    l_d_r_w_quat_data_stop_270.append(d_r_w_data_stop_270)
    
data_stop_270['e_r_x']=l_e_r_x_quat_data_stop_270
data_stop_270['e_r_y']=l_e_r_y_quat_data_stop_270
data_stop_270['e_r_z']=l_e_r_z_quat_data_stop_270
data_stop_270['e_r_w']=l_e_r_w_quat_data_stop_270
data_stop_270['d_r_x']=l_d_r_x_quat_data_stop_270
data_stop_270['d_r_y']=l_d_r_y_quat_data_stop_270
data_stop_270['d_r_z']=l_d_r_z_quat_data_stop_270
data_stop_270['d_r_w']=l_d_r_w_quat_data_stop_270



In [ ]:
data_stop_0_90 = data_stop_0.append(data_stop_90, ignore_index=True)
data_stop_0_90_180 = data_stop_0_90.append(data_stop_180, ignore_index=True)
data_stop = data_stop_0_90_180.append(data_stop_270, ignore_index=True)
data_stop

In [ ]:
data_0_slow= old_full_data.loc[old_full_data['speed']=="slow"][1:].reset_index(drop=True)
data_0_fast=old_full_data.loc[old_full_data['speed']=="fast"][1:].reset_index(drop=True)

In [ ]:
data_0 = data_0_slow.append(data_0_fast, ignore_index=True)
data_0

In [ ]:
data_90_left_foot_without_negativation = swap_columns(data_0, 'e_t_x', 'e_t_z')
data_90_left_foot_without_negativation["e_t_z"] = data_90_left_foot_without_negativation["e_t_z"]*-1
data_90_left_foot=data_90_left_foot_without_negativation
data_90_left_right_foot_without_negativation = swap_columns(data_90_left_foot, 'd_t_x', 'd_t_z')
data_90_left_right_foot_without_negativation["d_t_z"] = data_90_left_right_foot_without_negativation["d_t_z"]*-1
data_90=data_90_left_right_foot_without_negativation

data_90.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_90.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_90.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_90.rename(columns={'d_t_z':'d_t_x2'}, inplace=True)
data_90.rename(columns={'d_t_x':'d_t_z'}, inplace=True)
data_90.rename(columns={'d_t_x2':'d_t_x'}, inplace=True)
data_90

In [ ]:
data_180_left_foot_without_negativation = swap_columns(data_90, 'e_t_x', 'e_t_z')
data_180_left_foot_without_negativation["e_t_z"] = data_180_left_foot_without_negativation["e_t_z"]*-1
data_180_left_foot=data_180_left_foot_without_negativation
data_180_left_right_foot_without_negativation = swap_columns(data_180_left_foot, 'd_t_x', 'd_t_z')
data_180_left_right_foot_without_negativation["d_t_z"] = data_180_left_right_foot_without_negativation["d_t_z"]*-1
data_180=data_180_left_right_foot_without_negativation

data_180.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_180.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_180.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_180.rename(columns={'d_t_z':'d_t_x2'}, inplace=True)
data_180.rename(columns={'d_t_x':'d_t_z'}, inplace=True)
data_180.rename(columns={'d_t_x2':'d_t_x'}, inplace=True)
data_180

In [ ]:
data_270_left_foot_without_negativation = swap_columns(data_180, 'e_t_x', 'e_t_z')
data_270_left_foot_without_negativation["e_t_z"] = data_270_left_foot_without_negativation["e_t_z"]*-1
data_270_left_foot=data_270_left_foot_without_negativation
data_270_left_right_foot_without_negativation = swap_columns(data_270_left_foot, 'd_t_x', 'd_t_z')
data_270_left_right_foot_without_negativation["d_t_z"] = data_270_left_right_foot_without_negativation["d_t_z"]*-1
data_270=data_270_left_right_foot_without_negativation

data_270.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_270.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_270.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_270.rename(columns={'d_t_z':'d_t_x2'}, inplace=True)
data_270.rename(columns={'d_t_x':'d_t_z'}, inplace=True)
data_270.rename(columns={'d_t_x2':'d_t_x'}, inplace=True)
data_270

In [ ]:
l_e_r_x_quat_data_90=[]
l_e_r_y_quat_data_90=[]
l_e_r_z_quat_data_90=[]
l_e_r_w_quat_data_90=[]
l_d_r_x_quat_data_90=[]
l_d_r_y_quat_data_90=[]
l_d_r_z_quat_data_90=[]
l_d_r_w_quat_data_90=[]
for index_m_90, row_m_90 in data_90.iterrows():
    e_r_x_euler_data_90,e_r_y_euler_data_90,e_r_z_euler_data_90=euler_from_quaternion(row_m_90[3],row_m_90[4],row_m_90[5],row_m_90[6])
    d_r_x_euler_data_90,d_r_y_euler_data_90,d_r_z_euler_data_90=euler_from_quaternion(row_m_90[10],row_m_90[11],row_m_90[12],row_m_90[13])
    
    e_r_z_euler_data_90=e_r_z_euler_data_90+np.pi/2
    d_r_z_euler_data_90=d_r_z_euler_data_90+np.pi/2
    
    e_r_x_data_90, e_r_y_data_90, e_r_z_data_90,e_r_w_data_90=get_quaternion_from_euler(e_r_x_euler_data_90, e_r_y_euler_data_90,e_r_z_euler_data_90)
    d_r_x_data_90, d_r_y_data_90, d_r_z_data_90,d_r_w_data_90=get_quaternion_from_euler(d_r_x_euler_data_90, d_r_y_euler_data_90,d_r_z_euler_data_90)
    
    l_e_r_x_quat_data_90.append(e_r_x_data_90)
    l_e_r_y_quat_data_90.append(e_r_y_data_90)
    l_e_r_z_quat_data_90.append(e_r_z_data_90)
    l_e_r_w_quat_data_90.append(e_r_w_data_90)
    l_d_r_x_quat_data_90.append(d_r_x_data_90)
    l_d_r_y_quat_data_90.append(d_r_y_data_90)
    l_d_r_z_quat_data_90.append(d_r_z_data_90)
    l_d_r_w_quat_data_90.append(d_r_w_data_90)
    
data_90['e_r_x']=l_e_r_x_quat_data_90
data_90['e_r_y']=l_e_r_y_quat_data_90
data_90['e_r_z']=l_e_r_z_quat_data_90
data_90['e_r_w']=l_e_r_w_quat_data_90
data_90['d_r_x']=l_d_r_x_quat_data_90
data_90['d_r_y']=l_d_r_y_quat_data_90
data_90['d_r_z']=l_d_r_z_quat_data_90
data_90['d_r_w']=l_d_r_w_quat_data_90



In [ ]:
l_e_r_x_quat_data_180=[]
l_e_r_y_quat_data_180=[]
l_e_r_z_quat_data_180=[]
l_e_r_w_quat_data_180=[]
l_d_r_x_quat_data_180=[]
l_d_r_y_quat_data_180=[]
l_d_r_z_quat_data_180=[]
l_d_r_w_quat_data_180=[]
for index_m_180, row_m_180 in data_180.iterrows():
    e_r_x_euler_data_180,e_r_y_euler_data_180,e_r_z_euler_data_180=euler_from_quaternion(row_m_180[3],row_m_180[4],row_m_180[5],row_m_180[6])
    d_r_x_euler_data_180,d_r_y_euler_data_180,d_r_z_euler_data_180=euler_from_quaternion(row_m_180[10],row_m_180[11],row_m_180[12],row_m_180[13])
    
    e_r_z_euler_data_180=e_r_z_euler_data_180+np.pi/2
    d_r_z_euler_data_180=d_r_z_euler_data_180+np.pi/2
    
    e_r_x_data_180, e_r_y_data_180, e_r_z_data_180,e_r_w_data_180=get_quaternion_from_euler(e_r_x_euler_data_180, e_r_y_euler_data_180,e_r_z_euler_data_180)
    d_r_x_data_180, d_r_y_data_180, d_r_z_data_180,d_r_w_data_180=get_quaternion_from_euler(d_r_x_euler_data_180, d_r_y_euler_data_180,d_r_z_euler_data_180)
    
    l_e_r_x_quat_data_180.append(e_r_x_data_180)
    l_e_r_y_quat_data_180.append(e_r_y_data_180)
    l_e_r_z_quat_data_180.append(e_r_z_data_180)
    l_e_r_w_quat_data_180.append(e_r_w_data_180)
    l_d_r_x_quat_data_180.append(d_r_x_data_180)
    l_d_r_y_quat_data_180.append(d_r_y_data_180)
    l_d_r_z_quat_data_180.append(d_r_z_data_180)
    l_d_r_w_quat_data_180.append(d_r_w_data_180)
    
data_180['e_r_x']=l_e_r_x_quat_data_180
data_180['e_r_y']=l_e_r_y_quat_data_180
data_180['e_r_z']=l_e_r_z_quat_data_180
data_180['e_r_w']=l_e_r_w_quat_data_180
data_180['d_r_x']=l_d_r_x_quat_data_180
data_180['d_r_y']=l_d_r_y_quat_data_180
data_180['d_r_z']=l_d_r_z_quat_data_180
data_180['d_r_w']=l_d_r_w_quat_data_180



In [ ]:
l_e_r_x_quat_data_270=[]
l_e_r_y_quat_data_270=[]
l_e_r_z_quat_data_270=[]
l_e_r_w_quat_data_270=[]
l_d_r_x_quat_data_270=[]
l_d_r_y_quat_data_270=[]
l_d_r_z_quat_data_270=[]
l_d_r_w_quat_data_270=[]
for index_m_270, row_m_270 in data_270.iterrows():
    e_r_x_euler_data_270,e_r_y_euler_data_270,e_r_z_euler_data_270=euler_from_quaternion(row_m_270[3],row_m_270[4],row_m_270[5],row_m_270[6])
    d_r_x_euler_data_270,d_r_y_euler_data_270,d_r_z_euler_data_270=euler_from_quaternion(row_m_270[10],row_m_270[11],row_m_270[12],row_m_270[13])
    
    e_r_z_euler_data_270=e_r_z_euler_data_270+np.pi/2
    d_r_z_euler_data_270=d_r_z_euler_data_270+np.pi/2
    
    e_r_x_data_270, e_r_y_data_270, e_r_z_data_270,e_r_w_data_270=get_quaternion_from_euler(e_r_x_euler_data_270, e_r_y_euler_data_270,e_r_z_euler_data_270)
    d_r_x_data_270, d_r_y_data_270, d_r_z_data_270,d_r_w_data_270=get_quaternion_from_euler(d_r_x_euler_data_270, d_r_y_euler_data_270,d_r_z_euler_data_270)
    
    l_e_r_x_quat_data_270.append(e_r_x_data_270)
    l_e_r_y_quat_data_270.append(e_r_y_data_270)
    l_e_r_z_quat_data_270.append(e_r_z_data_270)
    l_e_r_w_quat_data_270.append(e_r_w_data_270)
    l_d_r_x_quat_data_270.append(d_r_x_data_270)
    l_d_r_y_quat_data_270.append(d_r_y_data_270)
    l_d_r_z_quat_data_270.append(d_r_z_data_270)
    l_d_r_w_quat_data_270.append(d_r_w_data_270)
    
data_270['e_r_x']=l_e_r_x_quat_data_270
data_270['e_r_y']=l_e_r_y_quat_data_270
data_270['e_r_z']=l_e_r_z_quat_data_270
data_270['e_r_w']=l_e_r_w_quat_data_270
data_270['d_r_x']=l_d_r_x_quat_data_270
data_270['d_r_y']=l_d_r_y_quat_data_270
data_270['d_r_z']=l_d_r_z_quat_data_270
data_270['d_r_w']=l_d_r_w_quat_data_270



In [ ]:
# data_stop.to_csv('stop_90_quat.csv', index=False, header=True)
# data_0.to_csv('0_90_quat.csv', index=False, header=True)
# data_90.to_csv('90_90_quat.csv', index=False, header=True)
# data_180.to_csv('180_90_quat.csv', index=False, header=True)
# data_270.to_csv('270_90_quat.csv', index=False, header=True)

In [ ]:
data_90